In [ ]:
#dimensions: 
#1: Time (day)
#2: Time (time of day - i.e. 1:00 pm) We can possibly do military time for this 
#3: Output (effectivness) between 1 and 4 

In [ ]:
#importing libraries so we can graph and do calculations 
#Import machine learning techniques as well 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor 
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.pipeline import make_pipeline 
import os 
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error 
import tensorflow as tf 
print(tf.__version__) 

In [ ]:
from __future__ import print_function

import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

ModuleNotFoundError: ignored

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('Final_Proj_Dataset(TimeAI)').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
df =  pd.DataFrame.from_records(rows) 

In [ ]:
df.columns = df.iloc[0]
df = df.iloc[1:].reset_index(drop=True)

In [ ]:
df['Task_Class'].value_counts()

In [ ]:
#Save the data columns in some variables
#for each row add it to the data 

# time = df[0].astype(float).to_numpy()
time = df['Time'].astype(float).to_numpy()
day = df['Day'].astype(float).to_numpy()
output = df['Time Taken Rating'].astype(float).to_numpy()

In [ ]:
#Have to create some type of legend to explore it and explain to users better what is going on 
#Need to label our points with coordinates so we can make sure our graph is set up correctly
#Why are the axis labels reversed? 
#Have to adjust output scales  

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.set_title('Day-time-TimeTakenOutput')
ax.set_xlabel('day')
ax.set_ylabel('time')
ax.set_zlabel('output')
# ax.set_xlim([0, 1])
# ax.set_ylim([0, 1])
# ax.set_zlim([0, 1])
ax.scatter(day, time, output, c=output, cmap='bwr')
plt.show()

In [ ]:
ax.plot_trisurf(day,time,output,cmap='bwr',edgecolor='none');
plt.show() 

In [ ]:
X = df[['Time','Output', 'Day','Task Time Length (in hours) ']]

In [ ]:
y = df['Time Taken Rating']

In [ ]:
Xtrain, Xtest,ytrain, ytest = train_test_split(X, y, test_size=0.30, random_state=0)

In [ ]:
Xtrain.shape

In [ ]:
Xtest.shape

In [ ]:
ytrain.shape

In [ ]:
ytest.shape

In [ ]:
model = LinearRegression() 

poly_model = make_pipeline(PolynomialFeatures(1), LinearRegression()) 
poly_model.fit(Xtrain, ytrain)
model.fit(Xtrain,ytrain)
lypred = model.predict(Xtest)
ypred = poly_model.predict(Xtest)
mae = mean_absolute_error(ytest,ypred)
mse = mean_squared_error(ytest, ypred)
lmae = mean_absolute_error(ytest,lypred)
lmse = mean_squared_error(ytest,lypred)
print('MAE: {:.4f}, MSE: {:.4f}'.format(mae,mse))
print('Linear Model: ', 'MAE: {:.4f}, MSE: {:.4f}'.format(lmae,lmse))

In [ ]:
pd.crosstab(df['Time Taken Rating'], df['Task Time Length (in hours) ']).plot.bar(figsize=(10,10))

Observations: 
- Output -> Time Taken Rating 
  - Changing the output variable from Output that is measured in 1-4 about general desired output to the time taken rating lowers the error on our prediction. 

  With Time, Day, Output, Task Time as training variables : 

Lowest Output as test variable: 22 degrees  
- MAE: 27% 
- MSE: 36% 


Lowest Time Taken Rating as test variable: 22 degrees  
- MAE: 40% 
- MSE: 66% 

- When tasks are shorter, it's easier to be efficient, I see from the crosstab that I have more tasks with a short time length and a good Task Time Length Rating

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
estimators = range(1,100)

for i in range(len(estimators)): 
  regressor = RandomForestRegressor(n_estimators=estimators[i],random_state=0) 
  regressor.fit(Xtrain, ytrain) 
  ypred = regressor.predict(Xtest)
  mae = mean_absolute_error(ytest,ypred)
  mse = mean_squared_error(ytest, ypred) 
  print('MAE: {:.4f}, MSE: {:.4f}'.format(mae,mse))


In [ ]:
newDf = df[['Time Taken Rating','Output', 'Task Time Length (in hours) ', 'Task_Class']]

In [ ]:
nowDf = pd.get_dummies(newDf) 
nowDf

In [ ]:
corr = nowDf.corr() 
corr

In [ ]:
sns.heatmap(corr)

In [ ]:
from keras.models import Sequential 
from keras.layers import Dense 
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder 

In [ ]:
lb = LabelEncoder() 
y = lb.fit_transform(y) 

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler() 
X_scale = min_max_scaler.fit_transform(X) 

In [ ]:
X_train, XvalTest, y_train, yvalTest = train_test_split(X_scale, y, test_size=0.3) 

In [ ]:
X_val, X_test, Y_val, Y_test = train_test_split(XvalTest,yvalTest, test_size=0.5) 

In [ ]:
annModel = Sequential([
        Dense(32, activation='relu', input_shape=(4,)), 
        Dense(32, activation='relu',),
        Dense(32, activation='relu',), 
        Dense(1, activation='sigmoid'), 
])

annModel.compile(optimizer='sgd', loss='binary_crossentropy', 
                 metrics=['accuracy']) 

In [ ]:
X_train

In [ ]:
hist = annModel.fit(X_train,y_train, batch_size=32, epochs=100, validation_data=(X_val,Y_val)) 


In [ ]:
annModel.summary()

In [ ]:
X_test

In [ ]:
y_pred = annModel.predict(X_test) 

In [ ]:
Y_test

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import accuracy_score 
annModel.evaluate(X_test,Y_test)[1]

In [ ]:
plt.plot(hist.history['loss']) 
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='lower right')
plt.show()